In [3]:
#=
N-Body Simulation With Euler Or Runge Kutta Method
=#
# Mass Of The Sun And Each Planet
#    Sun,      Mercury, Venus,    Earth,     Mars,     Jupiter, Saturn,   Uranus,  Neptune
m = [1.9891e30 3.301e23 4.8675e24 5.97237e24 6.4171e23 1.898e27 5.6834e26 8.681e25 1.02413e26]
# Distance From Sun
d = [0 5.790905e10 1.0820893e11 1.49598023e11 2.279392e11 7.7857e11 1.43353e12 2.870972e12 4.495e12]
# Velocity
v = [0 4.736e4 3.502e4 2.978e4 2.407e4 1.307e4 9.68e3 6.8e3 5.43e3]
# Degree Of Position In Orbit
degrees =  [0 -10 30 27 160 254 220 -43 267]
alpha = degrees * pi / 180

# How Many Planets Should Be In The Calculation
planets = 4

# Initialvalues
p0 = zeros(Float64,(planets + 1, 2))
v0 = zeros(Float64,(planets + 1, 2))
# Rotationmatrix
rotation(alpha) = [[cos(alpha) -sin(alpha)]; [sin(alpha) cos(alpha)]]
for i = 2:planets+1
    p0[i,:] = rotation(alpha[i]) * [d[i] 0]'
    v0[i,:] = rotation(alpha[i]) * [0 v[i]]'
end

# Newtons Gravitational Constant
G = 6.67e-11

# Forces From N Body Simulation
function dF(pos, vel)
    # Eulklidian Norm
    norm(x) = sqrt(x[1] * x[1] + x[2] * x[2])
    
    d_pos = vel
    d_vel = zeros(size(pos, 1), 2)

    for i = 1:size(pos, 1)
        s = zeros(2)
        for j = 1:size(pos, 1)
            if i != j
                d = pos[j,:] - pos[i,:]
                g = norm(pos[i,:] - pos[j,:])
                s += g >= 0.3 ? d * m[j] / (g^3) : d * m[j] / 0.027
            end
        end
        d_vel[i,:] = G * s
    end
    return d_pos, d_vel
end

# Explicit Euler Method of Order 1
function EE(p0, v0, dt, n)
    p = zeros(Float64,(size(p0,1), size(p0,2), n + 1))
    p[:,:,1] = p0
    v = v0
    for i = 1:n
        dp, dv = dF(p[:,:,i], v)
        p[:,:,i + 1] = p[:,:,i] + dt * dp
        v += dt * dv
    end
    return p
end

# Explicit Classical Runge Kutta Method of Order 4
function RK(p0, v0, dt, n)
    p = zeros(Float64,(size(p0, 1), size(p0, 2), n + 1))
    p[:,:,1] = p0
    v = v0
    for i = 1:n
        dp1, dv1 = dF(p[:,:,i], v)
        dp2, dv2 = dF(p[:,:,i] + dt / 2 * dp1, v + dt / 2 * dv1)
        dp3, dv3 = dF(p[:,:,i] + dt / 2 * dp2, v + dt / 2 * dv2)
        dp4, dv4 = dF(p[:,:,i] + dt * dp3, v + dt * dv3)

        p[:,:,i + 1] = p[:,:,i] + dt * (dp1 + 2 * dp2 + 2* dp3 + dp4) / 6
        v += dt * (dv1 + 2 * dv2 + 2 * dv3 + dv4) / 6
    end
    return p
end

# In Seconds
dt = 300
T = 3600 * 24 * 365.25 * 3

# Iterations
n = Int(floor(T/dt))
println("$n Iterations")

# Do iterations with Euler or Runge Kutta method
#@time p = EE(p0, v0, dt, n);
@time p = RK(p0, v0, dt, n);

315576 Iterations
 69.631033 seconds (307.69 M allocations: 23.898 GiB, 5.45% gc time, 1.49% compilation time)


In [ ]:
using Plots
# x and y limit
limit = maximum(abs.(p)) * 1.05

# Days Per Frame
days_per_frame = 3

step = Int(24 * 3600 / dt * days_per_frame)
println("Animation will be $((T/(3600*24))/(days_per_frame * 25)) seconds long")

anim = @animate for i = 1:step:n+1
    scatter(p[:,1,i], p[:,2,i],#=
        =#xlims=(-limit,limit), ylims=(-limit,limit), aspect_ratio=1, dpi=150,#=
        =#title="Day $(Int(round(i*dt/(3600*24),digits=0)))",#=
        =#xlabel="x", ylabel="y", label="Sun",#=
        =#mc = ([:yellow,:honeydew4,:coral,:royalblue4,:firebrick2,:orangered2,:darkorange1,:lightblue2,:dodgerblue1])[1:planets+1])
end

gif(anim, "NBodySim.gif",fps = 25)